<h2>Four channel Face output- Face extracted using dlib,openCV</h2>

In [13]:
import dlib
import cv2
#from imutils import face_utils
import numpy as np
#from PIL import Image

def rectangle(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    # return a tuple of (x, y, w, h) (x,y)->Top-left corner w->width of rect h->depth of rect
    return (x, y, w, h)



load = str(input("Enter the image name(Along with extension):\n"))
# load the input image, resize it, and convert it to grayscale
image = cv2.imread(load)

#convert from RBG to grayscale i.e.,PRE-PROCESS
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# dlib frontal face detector is used to find the faces
detector = dlib.get_frontal_face_detector()

#to predict the 68 landmark features on the face- from which we use first 17(jaw line) landmarks
predictor_path = "dlib\shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(predictor_path)

# detect faces in the grayscale image
face_rects = detector(gray, 1) 

#prints Number of detected faces
print("\nNumber of faces Detected:"+str(len(face_rects)))

# loop over the face detections
for (i, rect) in enumerate(face_rects):
    
    #returned values are stored in tuple top-left  corner and width and height
    (x1, y1, w, h) = rectangle(rect)
    
    #make a new black image on which we stick our cropped face this is of input image dimensions 
    out_face = np.zeros_like(image)
    
    
    #prints each detected face top-left corners and width and height
    print("Face:"+str(i+1))
    
    #landmark predictor given a image and a face
    shape = predictor(gray, rect)
    
    #finding the landmark points and manually give top two edges
    pts = np.empty([18, 2], dtype = int)
    for j in range(18):
        if j == 0:
            pts[j][0] = x1
            pts[j][1] = y1
        elif j == 17:
            pts[j][0] = x1+w
            pts[j][1] = y1
        else:
            pts[j][0] = shape.part(j).x
            pts[j][1] = shape.part(j).y
    
    #constructing a mask
    remapped_shape = np.zeros_like(shape)
    image_mask = np.zeros((image.shape[0],image.shape[1]))
    
    #array of landmark pts
    print('Facial Landmarks:')
    print(pts)
    
    #marking the crop region in mask
    cv2.fillPoly(image_mask,[np.array(pts, dtype=np.int32)],1)
    image_mask = image_mask.astype(np.bool)
    
    #inserting that mask on output image
    out_face[image_mask] = image[image_mask]
    
    #splitting the channels
    b_channel, g_channel, r_channel = cv2.split(out_face)
    
    #initializing alpha channel
    alpha_channel = np.zeros_like(gray)
    
    #assigning opaque value at image existing pixels
    for p in range(alpha_channel.shape[0]):
        for q in range(alpha_channel.shape[1]):
            if b_channel[p][q]!=0 or g_channel[p][q]!=0 or r_channel[p][q]!=0:
                alpha_channel[p][q] = 255
                
    #merging the four channels
    output_face = cv2.merge((b_channel, g_channel, r_channel, alpha_channel))
    
    #outputting the dark background face cropped image 
    string = "output"+str(i+1)+".png"
    cv2.imwrite(string, output_face)

Enter the image name(Along with extension):
gate.jpg

Number of faces Detected:1
Face:1
Facial Landmarks:
[[115 157]
 [112 238]
 [115 261]
 [120 285]
 [127 307]
 [141 326]
 [159 343]
 [181 356]
 [207 359]
 [233 356]
 [254 343]
 [270 326]
 [282 307]
 [288 286]
 [292 264]
 [297 241]
 [298 218]
 [301 157]]
